# A/B Testing: Evaluating UX Redesign on Conversion and Engagement Metrics

by Lau Wen Jun

## Disclaimer:
The dataset used in this analysis is a **synthetic dataset generated solely for demonstration purposes and does not reflect real-world user data**. You can download the dataset [here](https://www.dropbox.com/scl/fi/68qegh7dphc33gx6thrk5/ab_ux.xlsx?rlkey=585m46tw6i1cfbujbgxjgukid&st=8gxabc6g&dl=1).

## Table of Contents

1. [Business Context](#1.-Business-Context)
2. [Key Variables](#2.-Key-Variables)
3. [Objective](#3.-Objective)
4. [Experimental Design](#4.-Experimental-Design)
5. [Statistical Tests](#5.-Statistical-Tests)
6. [Results](#6.-Results)
7. [Conclusions](#7.-Conclusions)

## [1. Business Context](#Table-of-Contents)



The company implemented a redesigned product page aimed at enhancing:

- Usability and user experience

- Mobile responsiveness across devices

- Visual appeal and content clarity

Before rolling out the redesign to all users, the team conducted an A/B test to assess whether the new layout led to measurable improvements in user behavior and engagement.

## [2. Key Variables](#Table-of-Contents)

The A/B test focused on evaluating the following performance indicators:

- **Conversion Rate (CR):** Percentage of sessions that resulted in a purchase

- **Bounce Rate (BR):** Percentage of sessions where users left the page without any interaction

- **Average Time on Page:** The mean time (in seconds) users spent on the product page

These metrics were chosen to capture both transactional effectiveness and overall user engagement with the new design.

## [3. Objective](#Table-of-Contents)

To evaluate whether a redesigned product page layout improves user engagement and conversion behavior compared to the original layout, using controlled A/B testing.

## [4. Experimental Design](#Table-of-Contents)

The A/B test compared two versions of a product page:

- Control Group (Version A): Exposed to the current layout.

- Test Group (Version B): Exposed to the redesigned layout.

Users were **randomly assigned** at the session level, ensuring each visit had an equal chance of being exposed to either version. The traffic was **evenly split (50/50)** using a backend feature flag, ensuring consistent experiences per session.

To maintain test integrity:

- The redesign was deployed silently (no banners or prompts), so users were unaware they were part of a test.

- All other variables (pricing, recommendations, functionality) were held constant.

- Data was collected passively via analytics logs over the same fixed time period.

This setup ensured that any observed changes could be attributed to the UX redesign alone.

## [5. Statistical Tests](#Table-of-Contents)

#### 🔹 Conversion Rate (CR)
$$
CR = \frac{\text{Conversions}}{\text{Sessions}} \times 100
$$

**Z-test for proportions**:

Used for comparing **conversion rate** and **bounce rate**, which are binary outcomes (converted vs. not converted, bounced vs. not bounced).

$$
z = \frac{p_1 - p_2}{\sqrt{p(1 - p)\left( \frac{1}{n_1} + \frac{1}{n_2} \right)}}
$$

Where:

- \( $p_1$, $p_2$ \): conversion rates in test and control groups  
- \( $n_1$, $n_2$ \): sample sizes  
- \( $p$ \): pooled rate, calculated as:

$$
p = \frac{x_1 + x_2}{n_1 + n_2}
$$
---

#### 🔹 Bounce Rate (BR)
$$
BR = \frac{\text{Bounces}}{\text{Sessions}} \times 100
$$

Tested using the **same z-test for proportions**.

---

#### 🔹 Average Time on Page

**Independent t-test**:

Used for comparing **average time** on page, a continuous metric.

$$
t = \frac{\bar{x}_1 - \bar{x}_2}
{\sqrt{\frac{s_1^2}{n_1} + \frac{s_2^2}{n_2}}}
$$

Where:
- \( $\bar{x}_1$, $\bar{x}_2$ \): mean time on page
- \( $s_1^2$, $s_2^2$ \): variances
- \( $n_1$, $n_2$ \): sample sizes

## [6. Results](#Table-of-Contents)

The following results were obtained using a custom Python script that calculated group-level metrics, percentage lifts, and statistical significance using pandas, scipy, and statsmodels.



In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest

# Load data
url = "https://www.dropbox.com/scl/fi/68qegh7dphc33gx6thrk5/ab_ux.xlsx?rlkey=585m46tw6i1cfbujbgxjgukid&st=8gxabc6g&dl=1"
df = pd.read_excel(url)

# Grouped metrics
summary = df.groupby("group").agg(
    sessions=('converted', 'count'),
    conversions=('converted', 'sum'),
    bounces=('bounce', 'sum'),
    avg_time_on_page=('time_on_page', 'mean')
).reset_index()

summary['conversion_rate'] = summary['conversions'] / summary['sessions']
summary['bounce_rate'] = summary['bounces'] / summary['sessions']
summary['avg_time_on_page'] = summary['avg_time_on_page'].round(2)

# Split into control and test
control = summary[summary['group'] == 'control'].iloc[0]
test = summary[summary['group'] == 'test'].iloc[0]

# Calculate lift
lift_cr = ((test['conversion_rate'] - control['conversion_rate']) / control['conversion_rate']) * 100
lift_br = ((test['bounce_rate'] - control['bounce_rate']) / control['bounce_rate']) * 100
lift_time = ((test['avg_time_on_page'] - control['avg_time_on_page']) / control['avg_time_on_page']) * 100

# Statistical tests
z_cr, p_cr = proportions_ztest([test['conversions'], control['conversions']],
                               [test['sessions'], control['sessions']])
z_br, p_br = proportions_ztest([test['bounces'], control['bounces']],
                               [test['sessions'], control['sessions']])
t_time, p_time = ttest_ind(
    df[df['group'] == 'test']['time_on_page'],
    df[df['group'] == 'control']['time_on_page'],
    equal_var=False
)

# Create consolidated summary table
table = pd.DataFrame({
    'Metric': ['Conversion Rate', 'Bounce Rate', 'Time on Page (s)'],
    'Control': [f"{control['conversion_rate']*100:.1f}%", f"{control['bounce_rate']*100:.1f}%", f"{control['avg_time_on_page']:.2f}"],
    'Test': [f"{test['conversion_rate']*100:.1f}%", f"{test['bounce_rate']*100:.1f}%", f"{test['avg_time_on_page']:.2f}"],
    'Lift (%)': [f"{lift_cr:+.2f}%", f"{lift_br:+.2f}%", f"{lift_time:+.2f}%"],
    'p-value': [f"{p_cr:.4f}", f"{p_br:.4f}", f"{p_time:.4f}"]
})

The A/B test compared user behavior across control and test groups using three key metrics: Conversion Rate, Bounce Rate, and Average Time on Page. The results are summarized in the table below:

In [ ]:
print("📊 UX A/B Test Summary Table:")
print(table.to_string(index=False))

📊 UX A/B Test Summary Table:
          Metric Control  Test Lift (%) p-value
 Conversion Rate    6.4%  6.6%   +3.13%  0.8560
     Bounce Rate   44.9% 34.8%  -22.49%  0.0000
Time on Page (s)   44.43 52.38  +17.89%  0.0000


- **Conversion Rate:** Increased slightly in the test group (+3.13%), but this difference was not statistically significant (p = 0.8560), suggesting the uplift may be due to chance.

- **Bounce Rate:** Dropped significantly in the test group (−22.72%), with a highly significant result (p < 0.001), indicating improved user retention and interest.

- **Time on Page:** Increased by 17.87% in the test group, with a statistically significant result (p < 0.001), suggesting the redesigned layout successfully encouraged deeper user engagement.

## [7. Conclusions](#Table-of-Contents)

The UX redesign significantly improved user engagement, with a lower bounce rate and longer time on page, but did not result in a statistically significant increase in conversions. Given these findings, the new design should be rolled out to all users, while continuing to monitor post-launch funnel metrics such as add-to-cart and checkout rates. Future A/B tests should focus on optimizing conversion-related elements like visual hierarchy and button design.